In [1]:
!nvidia-smi

Tue Jul 18 08:58:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:1B:00.0 Off |                    0 |
| N/A   33C    P0    41W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from pynvml import *
import torch.optim as optim


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    


In [3]:
import torch

print_gpu_utilization()

GPU memory occupied: 257 MB.


In [4]:
model_name = "T5-small-fine-tuned-Spider"
from transformers import AutoTokenizer, T5ForConditionalGeneration

[2023-07-18 08:58:13,066] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-18 08:58:15.372088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 08:58:15.521314: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-18 08:58:15.561356: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-18 08:58:17.096182: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [7]:
import pandas as pd
from datasets import Dataset


In [8]:
df1 = pd.read_csv('Training_set_IMDB/training_set_sample_no_target.csv')
df1 = df1.dropna()


train_df = df1.sample(frac=1, random_state=42)  # Set random_state for reproducibility


text = []
sql = []

for index, row in train_df.iterrows():
    text_input = "Translate to SQL: " + row['Text'],
    sql_input = row['SQL']
    text.append(text_input)
    sql.append(sql_input)
    
inputs = {"inputs": text,
        "target": sql}

train_dataset = Dataset.from_dict(inputs)

In [9]:
train_dataset

Dataset({
    features: ['inputs', 'target'],
    num_rows: 43063
})

In [10]:
def map_to_lenght(x):
    x["input_len"] = len(tokenizer(x["inputs"]).input_ids)
    x["input_longer_256"] = int(x["input_len"]>256)
    x["input_longer_128"] = int(x["input_len"]>128)
    x["input_longet_64"] = int(x["input_len"]>64)
    x["output_len"] = len(tokenizer(x["target"]).input_ids)
    x["output_longet_256"] = int(x["output_len"]>256)
    x["output_longet_128"] = int(x["output_len"]>128)
    x["output_longet_64"] = int(x["output_len"]>64)
    return x

sample_size = 20000
data_stats = train_dataset.select(range(sample_size)).map(map_to_lenght, num_proc=4)

Map (num_proc=4):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [11]:
def compute_and_print(x):
    if len(x["input_len"])==sample_size:
        print(
            f"Input mean: {sum(x['input_len'])/sample_size} \n % of input len > 256: {sum(x['input_longer_256'])/sample_size}, \n % of input len > 128: {sum(x['input_longer_128'])/sample_size}, \n % of input len > 64: {sum(x['input_longet_64'])/sample_size}, \n Ouput mean: {sum(x['output_len'])/sample_size},\n% of output len > 256: {sum(x['output_longet_256'])/sample_size}, \n% of output len > 128: {sum(x['output_longet_128'])/sample_size}, \n% of output len > 64: {sum(x['output_longet_64'])/sample_size}")

output = data_stats.map(compute_and_print, batched=True, batch_size=-1)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Input mean: 1.0 
 % of input len > 256: 0.0, 
 % of input len > 128: 0.0, 
 % of input len > 64: 0.0, 
 Ouput mean: 106.96725,
% of output len > 256: 0.0, 
% of output len > 128: 0.29155, 
% of output len > 64: 0.73015


In [12]:
def convert_to_features(example_batch, padding = "max_length"):
    inputs = tokenizer.batch_encode_plus(example_batch["inputs"],is_split_into_words=True, max_length=64, padding="max_length", truncation=True)
    
    targets = tokenizer.batch_encode_plus(example_batch["target"], max_length=256,padding = 'max_length',truncation = True)
    if padding == "max_length":
        targets["inputs_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in target] for target in targets["input_ids"]
        ]
    
    inputs["labels"] = targets['input_ids']
    return inputs

train_data = train_dataset.map(convert_to_features, batched=True, remove_columns=train_dataset.column_names)

Map:   0%|          | 0/43063 [00:00<?, ? examples/s]

In [13]:
train_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 43063
})

In [14]:
columns = ['input_ids', 'attention_mask', 'labels']

train_data.set_format(type='torch', columns=columns)

In [16]:
def sequence_check(list_of_seq):
    len_seq = len(list_of_seq[0])
    for seq in list_of_seq:
        if len(seq)!=len_seq:
            print(len(seq))
            return False
        
    return True

In [17]:
sequence_check(train_data['input_ids'])

True

In [18]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import os
output_dir = 'T5-Finetuned-with-IMDB-Spider'
os.mkdir(output_dir)



In [22]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    predict_with_generate=True,
    do_train=True,
    learning_rate=2e-4,
    weight_decay=0.1,
    logging_strategy="epoch",
    overwrite_output_dir=True,
    save_total_limit=3,
    push_to_hub=False,
    fp16=True,
    optim='adamw_torch'
)

In [23]:
from evaluate import load
rouge = load("rouge")

In [24]:
import numpy as np

def compute_metrics(pred):
    predictions, labels = pred
    
    pred_str = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    rouge_output = rouge.compute(predictions=pred_str, references=label_str)
    return {
        "rouge1": round(rouge_output['rouge1']* 100,4),
        "rouge2": round(rouge_output['rouge2']*100, 4),
        "rougeL": round(rouge_output['rougeL']*100, 4)
    }

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100)

In [27]:
trainer = Seq2SeqTrainer(
    model = model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
2692,0.031300
5384,0.008600


In [ ]:
trainer.save_model(output_dir)

In [ ]:
tokenizer.save_pretrained(output_dir)